<center>
    <p style="text-align:center">
        <img alt="arize llama-index logos" src="https://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/llama-index-knowledge-base-tutorial/arize_llamaindex.png" width="400">
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Evaluating and Improving a LlamaIndex Search and Retrieval Application</h1>

Imagine you're an engineer at Arize AI and you've built and deployed a documentation question-answering service using LlamaIndex. Users send questions about Arize's core product via a chat interface, and your service retrieves documents from your documentation in order to generate a response to the user. As the engineer in charge of evaluating and maintaining this system, you want to evaluate the quality of the responses from your service.

Phoenix helps you:
- identify gaps in your documentation
- detect queries for which the LLM gave bad responses
- detect failures to retrieve relevant context

In this tutorial, you will:

- Download an pre-indexed knowledge base of the Arize documentation and run a LlamaIndex application
- Visualize user queries and knowledge base documents to identify areas of user interest not answered by your documentation
- Find clusters of responses with negative user feedback
- Identify failed retrievals using cosine similarity, Euclidean distance, and LLM-assisted ranking metrics

Parts of this notebook require an [OpenAI API key](https://platform.openai.com/account/api-keys) to run. If you don't have an OpenAI key, you can still run Phoenix by skipping cells preceded by the 💭 emoji.


## Chatbot Architecture

Your chatbot was built using LlamaIndex's low-level API. The architecture of your chatbot is shown below and can be explained in five steps.

![llama-index chatbot architecture](http://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/llama-index-knowledge-base-tutorial/llama_index_chatbot_architecture.png)

1. The user sends a query about Arize to your service.
1. `langchain.embeddings.OpenAIEmbeddings` makes a request to the OpenAI embeddings API to embed the user query using the `text-embedding-ada-002` model.
1. `llama_index.retrievers.RetrieverQueryEngine` does a similarity search against the entries of your index knowledge base for the two most similar pieces of context by cosine similarity.
1. `llama_index.indices.query.ResponseSynthesizer` generates a response by formatting the query and retrieved context into a single prompt and sending a request to OpenAI chat completions API with the `gpt-3.5-turbo`.
2. The response is returned to the user.

Phoenix makes your search and retrieval system *observable* by capturing the inputs and outputs of these steps for analysis, including:

- your query embeddings
- the retrieved context and similarity scores for each query
- the generated response that is return to the user

With that overview in mind, let's dive into the notebook.

## 1. Install Dependencies and Import Libraries

Install Phoenix and LlamaIndex.

In [1]:
%pip install -Uq "arize-phoenix[evals,embeddings]" "openai>=1" "llama-index<0.10.0" gcsfs 'httpx<0.28'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.3/661.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 

Import libraries.

In [3]:
import os
import textwrap
from datetime import timedelta
from getpass import getpass

import numpy as np
import pandas as pd
from gcsfs import GCSFileSystem
from IPython.display import YouTubeVideo
from llama_index import (
    ServiceContext,
    StorageContext,
    load_index_from_storage,
)
from llama_index.callbacks import CallbackManager, OpenInferenceCallbackHandler
from llama_index.callbacks.open_inference_callback import as_dataframe
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.graph_stores.simple import SimpleGraphStore
from llama_index.llms import OpenAI

import phoenix as px
from phoenix.evals.retrievals import (
    classify_relevance,
    compute_precisions_at_k,
)

pd.set_option("display.max_colwidth", 1000)

## 2. Configure Your OpenAI API Key

💭 Configure your OpenAI API key.

In [4]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

🔑 Enter your OpenAI API key: ··········


## 3. Download Your Knowledge Base

Download your pre-built index from cloud storage and instantiate your storage context.

In [5]:
file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-phoenix-assets/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
    graph_store=SimpleGraphStore(),  # prevents unauthorized request to GCS
)

Download and unzip a pre-built knowledge base index consisting of chunks of the Arize documentation.

## 4. Run Your Question-Answering Service

💭 Start a LlamaIndex application from your downloaded index. Use the `OpenInferenceCallbackHandler` to store your data in [OpenInference format](https://github.com/Arize-ai/openinference), an open standard for capturing and storing AI model inferences that enables production LLMapp servers to seamlessly integrate with LLM observability solutions such as Arize and Phoenix.

In [6]:
callback_handler = OpenInferenceCallbackHandler()
service_context = ServiceContext.from_defaults(
    llm=OpenAI(model_name="gpt-3.5-turbo", temperature=0),
    embed_model=OpenAIEmbedding(model="text-embedding-ada-002"),
    callback_manager=CallbackManager(handlers=[callback_handler]),
)
index = load_index_from_storage(
    storage_context,
    service_context=service_context,
)
query_engine = index.as_query_engine()

💭 Ask a few questions of your question-answering service and view the responses.

In [7]:
max_line_length = 80
for query in [
    "How do I get an Arize API key?",
    "Can I create monitors with an API?",
    "How do I need to format timestamps?",
    "What is the price of the Arize platform",
]:
    print("Query")
    print("=====")
    print()
    print(textwrap.fill(query, max_line_length))
    print()
    response = query_engine.query(query)
    print("Response")
    print("========")
    print()
    print(textwrap.fill(str(response), max_line_length))
    print()

Query
=====

How do I get an Arize API key?

Response

Click the 'Get Your API Key' button on the top right of the Explorer page to
generate your API key.

Query
=====

Can I create monitors with an API?

Response

Yes, you can create monitors using the public-facing graphQL API provided.

Query
=====

How do I need to format timestamps?

Response

You need to format timestamps as an integer representing the UNIX Timestamp in
seconds.

Query
=====

What is the price of the Arize platform

Response

The price of the Arize platform is not specified in the provided context
information. For detailed pricing information, it is recommended to reach out to
the sales team at contacts@arize.com.



## 5. Load Your Data Into Pandas Dataframes

To use Phoenix, you must load your data into pandas dataframes.

💭 Your query data is saved in a buffer on the callback handler you defined in step 4. Load the data from the buffer into a dataframe.

In [8]:
query_data_buffer = callback_handler.flush_query_data_buffer()
sample_query_df = as_dataframe(query_data_buffer)
sample_query_df

,:id.id:,:timestamp.iso_8601:,:feature.text:prompt,:feature.[float].embedding:prompt,:prediction.text:response,:feature.[str].retrieved_document_ids:prompt,:feature.[float].retrieved_document_scores:prompt
0,deea6441-049d-4cea-8be0-d0fd0610f5a5,2025-05-01T20:20:37.037957,How do I get an Arize API key?,"[0.007313841488212347, -0.010544062592089176, -0.022766713052988052, -0.024797942489385605, -0.014768741093575954, 0.02588408812880516, 0.005769259762018919, 0.0013541538501158357, -0.014098716899752617, -0.03303571417927742, 0.018972260877490044, -0.007483110763132572, -0.0008181346347555518, 0.0028899195604026318, -0.02560197189450264, 0.01569972187280655, 0.016010047867894173, 0.007440793327987194, 0.0341077521443367, -0.007074043620377779, -0.02080601081252098, 0.0012421893188729882, 0.015008538961410522, -0.010875548236072063, -0.014352620579302311, 0.013351110741496086, -0.0021952104289084673, 0.004422158934175968, 0.003653394291177392, -0.028084587305784225, 0.03125838562846184, 0.022258903831243515, -0.0008401749073527753, -0.003401253605261445, -0.03306392580270767, 0.0050146011635661125, 0.019762182608246803, 0.01008562557399273, 0.006079586688429117, -0.0010050360579043627, 0.02117276005446911, -0.01826697215437889, 0.013153630308806896, -0.012236755341291428, -0.0072715...",assistant: Click the 'Get Your API Key' button on the top right of the Explorer page to generate your API key.,"[7f0e9f73fc9208f158db2bbb815ef2281b59411c9678608cfe429e5c5bfec217, 5d9c75f5fa537e3fdb16c9c0cd4b14aa65c556f1775102240710cd7a74e7f197]","[0.8830837164630378, 0.8669015303926858]"
1,64a87ce8-cb52-4b49-b407-36daa5cf87bd,2025-05-01T20:20:39.317035,Can I create monitors with an API?,"[-0.006982418708503246, -0.0234614796936512, 0.0004936801851727068, -0.02667083404958248, -0.0032525828573852777, 0.009475892409682274, -0.022327139973640442, -0.004077400546520948, -0.0024865572340786457, -0.012775163166224957, 0.011709989979863167, 0.005720118992030621, 0.005429617594927549, 0.008085633628070354, -0.023115644231438637, 0.016019100323319435, 0.021953638643026352, 0.016295768320560455, 0.014787926338613033, -0.02207813784480095, -0.012539994902908802, 0.009302974678575993, 0.004921239335089922, 0.003569022286683321, -0.026795335114002228, 0.01084540132433176, 0.0001152064505731687, -0.015604098327457905, -0.029382184147834778, -0.00267849606461823, 0.04374127462506294, -0.00503190653398633, -0.014677259139716625, -0.02263147570192814, -0.014787926338613033, 0.0006946970825083554, 0.021483302116394043, 0.025149157270789146, 0.013446084223687649, 0.004108525812625885, 0.02253464050590992, 0.015451930463314056, 0.016932105645537376, -0.006083246320486069, 0.0011862157...","assistant: Yes, you can create monitors using the public-facing graphQL API provided.","[7ca1fc69a37f7f6f2589a000dd41fd455489547309b40bd5ec005acc6e242f96, a4fe23175b4179e6f83bb71ee248fd1c66f7146ba22b53786536e6de6ffefc39]","[0.8425179464267444, 0.8219968931201883]"
2,d06c6ae7-5605-4b17-92dc-9b3ba34c0582,2025-05-01T20:20:40.538349,How do I need to format timestamps?,"[-0.015807045623660088, 0.008870591409504414, -0.005659785121679306, -0.043051935732364655, -0.0045883567072451115, -0.009726341813802719, -0.003464748617261648, -0.015111313201487064, -0.015987936407327652, -0.008599255234003067, 0.015278289094567299, 0.01618274115025997, -0.01417207345366478, -0.005743273068219423, -0.01817253604531288, 0.0019880561158061028, 0.009914190508425236, 0.013608530163764954, 0.024350643157958984, -0.021915577352046967, -0.026256950572133064, 0.01469387300312519, 0.009754171594977379, -0.006964283529669046, 0.0074443393386900425, -0.0009044524049386382, -0.0019097861368209124, -0.006226806901395321, -0.0016288839979097247, -0.0038439228665083647, 0.013858994469046593, -0.010797770693898201, -0.001970662735402584, -0.02913728356361389, -0.017908157780766487, -0.011458716355264187, -0.010456861928105354, -0.01375463418662548, 0.02259739674627781, -8.89776783878915e-05, -0.0010696889366954565, -

The columns of the dataframe are:

- **:id.id:**: the query ID
- **:timestamp.iso_8601:**: the time at which the query was made
- **:feature.text:prompt**: the query text
- **:feature.[float].embedding:prompt**: the embedding representation of the query
- **:prediction.text:response**: the final response presented to the user
- **:feature.[str].retrieved_document_ids:prompt**: the list of IDs of the retrieved documents
- **:feature.[float].retrieved_document_scores:prompt**: the lists of cosine similarities between the query and retrieved documents

The column names are in OpenInference format and describe the category, data type and intent of each column.

Running queries against a large dataset takes a long time. Download a dataframe containing query data.

In [ ]:
query_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/llama-index/arize-docs/query_data_complete3.parquet",
)
query_df.head()

In addition to the columns of the previous dataframe, this data has a few additional fields:

- **:tag.float:user_feedback:** approval or rejection from the user (-1 means thumbs down, +1 means thumbs up)
- **:tag.str:openai_relevance_0:** a binary classification (relevant vs. irrelevant) by GPT-4 predicting whether the first retrieved document is relevant to the query
- **:tag.str:openai_relevance_1:** a binary classification (relevant vs. irrelevant) by GPT-4 predicting whether the second retrieved document is relevant to the query

We'll go over how to compute the relevance classifications in section 6.

Next load your knowledge base into a dataframe.

In [ ]:
def storage_context_to_dataframe(storage_context: StorageContext) -> pd.DataFrame:
    """Converts the storage context to a pandas dataframe.

    Args:
        storage_context (StorageContext): Storage context containing the index
        data.

    Returns:
        pd.DataFrame: The dataframe containing the index data.
    """
    document_ids = []
    document_texts = []
    document_embeddings = []
    docstore = storage_context.docstore
    vector_store = storage_context.vector_store
    for node_id, node in docstore.docs.items():
        document_ids.append(node.hash)  # use node hash as the document ID
        document_texts.append(node.text)
        document_embeddings.append(np.array(vector_store.get(node_id)))
    return pd.DataFrame(
        {
            "document_id": document_ids,
            "text": document_texts,
            "text_vector": document_embeddings,
        }
    )


database_df = storage_context_to_dataframe(storage_context)
database_df = database_df.drop_duplicates(subset=["text"])
database_df.head()

The columns of your dataframe are:

- **document_id:** the ID of the chunked document
- **text:** the chunked text in your knowledge base
- **text_vector:** the embedding vector for the text, computed during the LlamaIndex build using "text-embedding-ada-002" from OpenAI

The query and database datasets are drawn from different distributions; the queries are short questions while the database entries are several sentences to a paragraph. The embeddings from OpenAI's "text-embedding-ada-002" capture these differences and naturally separate the query and context embeddings into distinct regions of the embedding space. When using Phoenix, you want to "overlay" the query and context embedding distributions so that queries appear close to their retrieved context in the Phoenix point cloud. To achieve this, we compute a centroid for each dataset that represents an average point in the embedding distribution and center the two distributions so they overlap.

In [ ]:
database_centroid = database_df["text_vector"].mean()
database_df["text_vector"] = database_df["text_vector"].apply(lambda x: x - database_centroid)

query_centroid = query_df[":feature.[float].embedding:prompt"].mean()
query_df[":feature.[float].embedding:prompt"] = query_df[":feature.[float].embedding:prompt"].apply(
    lambda x: x - query_centroid
)

## 6. Run LLM-Assisted Evaluations

Cosine similarity and Euclidean distance are reasonable proxies for retrieval quality, but they don't always work perfectly. A novel idea is to use LLMs to measure retrieval quality by simply asking the LLM whether each retrieved document is relevant to the corresponding query.

💭 Use OpenAI to predict whether each retrieved document is relevant or irrelevant to the query.

⚠️ It's strongly recommended to use GPT-4 for evaluations if you have access.

In [ ]:
evals_model_name = "gpt-3.5-turbo"
# evals_model_name = "gpt-4"  # use GPT-4 if you have access
query_texts = sample_query_df[":feature.text:prompt"].tolist()
list_of_document_id_lists = sample_query_df[":feature.[str].retrieved_document_ids:prompt"].tolist()
document_id_to_text = dict(zip(database_df["document_id"].to_list(), database_df["text"].to_list()))

first_document_texts, second_document_texts = [
    [
        document_id_to_text[document_ids[document_index]]
        for document_ids in list_of_document_id_lists
    ]
    for document_index in [0, 1]
]
first_document_relevances, second_document_relevances = [
    [
        classify_relevance(query_text, document_text, evals_model_name)
        for query_text, document_text in zip(query_texts, first_document_texts)
    ]
    for document_texts in [first_document_texts, second_document_texts]
]


sample_query_df = sample_query_df.assign(
    retrieved_document_text_0=first_document_texts,
    retrieved_document_text_1=second_document_texts,
    relevance_0=first_document_relevances,
    relevance_1=second_document_relevances,
)
sample_query_df[
    [
        ":feature.text:prompt",
        "retrieved_document_text_0",
        "retrieved_document_text_1",
        "relevance_0",
        "relevance_1",
    ]
].rename(columns={":feature.text:prompt": "query_text"})

## 7. Compute Ranking Metrics

Now that you know whether each piece of retrieved context is relevant or irrelevant to the corresponding query, you can compute precision@k for k = 1, 2 for each query. This metric tells you what percentage of the retrieved context is relevant to the corresponding query.

precision@k = (# of top-k retrieved documents that are relevant) / (k retrieved documents)

If your precision@2 is greater than zero for a particular query, your LlamaIndex application successfully retrieved at least one relevant piece of context with which to answer the query. If the precision@k is zero for a particular query, that means that no relevant piece of context was retrieved.

Compute precision@k for k = 1, 2 and view the results.

In [ ]:
first_document_relevances = [
    {"relevant": True, "irrelevant": False}.get(rel)
    for rel in query_df[":tag.str:openai_relevance_0"].tolist()
]
second_document_relevances = [
    {"relevant": True, "irrelevant": False}.get(rel)
    for rel in query_df[":tag.str:openai_relevance_1"].tolist()
]
list_of_precisions_at_k_lists = [
    compute_precisions_at_k([rel0, rel1])
    for rel0, rel1 in zip(first_document_relevances, second_document_relevances)
]
precisions_at_1, precisions_at_2 = [
    [precisions_at_k[index] for precisions_at_k in list_of_precisions_at_k_lists]
    for index in [0, 1]
]
query_df[":tag.float:openai_precision_at_1"] = precisions_at_1
query_df[":tag.float:openai_precision_at_2"] = precisions_at_2
query_df[
    [
        ":tag.str:openai_relevance_0",
        ":tag.str:openai_relevance_1",
        ":tag.float:openai_precision_at_1",
        ":tag.float:openai_precision_at_2",
    ]
]

## 7. Launch Phoenix

Define your knowledge base dataset with a schema that specifies the meaning of each column (features, predictions, actuals, tags, embeddings, etc.). See the [docs](https://docs.arize.com/phoenix/) for guides on how to define your own schema and API reference on `phoenix.Schema` and `phoenix.EmbeddingColumnNames`.

In [ ]:
# get a random sample of 500 documents (including retrieved documents)
# this will be handled by by the application in a coming release
num_sampled_point = 500
retrieved_document_ids = set(
    [
        doc_id
        for doc_ids in query_df[":feature.[str].retrieved_document_ids:prompt"].to_list()
        for doc_id in doc_ids
    ]
)
retrieved_document_mask = database_df["document_id"].isin(retrieved_document_ids)
num_retrieved_documents = len(retrieved_document_ids)
num_additional_samples = num_sampled_point - num_retrieved_documents
unretrieved_document_mask = ~retrieved_document_mask
sampled_unretrieved_document_ids = set(
    database_df[unretrieved_document_mask]["document_id"]
    .sample(n=num_additional_samples, random_state=0)
    .to_list()
)
sampled_unretrieved_document_mask = database_df["document_id"].isin(
    sampled_unretrieved_document_ids
)
sampled_document_mask = retrieved_document_mask | sampled_unretrieved_document_mask
sampled_database_df = database_df[sampled_document_mask]

In [ ]:
database_schema = px.Schema(
    prediction_id_column_name="document_id",
    prompt_column_names=px.EmbeddingColumnNames(
        vector_column_name="text_vector",
        raw_data_column_name="text",
    ),
)
database_ds = px.Inferences(
    dataframe=sampled_database_df,
    schema=database_schema,
    name="database",
)

Define your query dataset. Because the query dataframe is in OpenInference format, Phoenix is able to infer the meaning of each column without a user-defined schema by using the `phoenix.Dataset.from_open_inference` class method.

In [ ]:
query_ds = px.Inferences.from_open_inference(query_df)

Launch Phoenix. Follow the instructions in the cell output to open the Phoenix UI.

In [ ]:
(session := px.launch_app(primary=query_ds, corpus=database_ds)).view()

## 8. Surface Problematic Clusters and Data Points

Phoenix helps you:

- visualize your embeddings
- color the resulting point cloud using evaluation metrics
- cluster the points and surface up problematic clusters based on whatever metric you care about

Follow along with the tutorial walkthrough [here](https://youtu.be/hbQYDpJayFw?t=1782), or view the video in your notebook by running the cell below. The video will show you how to investigate your query and knowledge base and identify problematic clusters of data points using Phoenix.

In [ ]:
start_time_in_seconds = int(timedelta(hours=0, minutes=29, seconds=42).total_seconds())
YouTubeVideo("hbQYDpJayFw", start=start_time_in_seconds, width=560, height=315)

Congrats! You've identified a problematic cluster of queries. You now have tools at your disposal to investigate clusters of queries where your search and retrieval application is performing poorly based on:

- query percentage
- user feedback
- LLM-assisted ranking metrics

As an actionable next step, you should augment your knowledge base to include information about the pricing and cost of the Arize platform, since your users seem especially interested in this topic.